In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML: MLOps阶段7：监控：面向AutoML表格模型的Vertex AI模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_automl.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td> 
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何在AutoML表格模型中使用Vertex AI模型监控。

### 目标

在这份笔记本中，您将学习如何使用`Vertex AI模型监控`服务来检测AutoML表格模型中输入预测请求中的特征偏差和漂移。

本教程使用以下Google Cloud ML服务：

- `AutoML`
- `Vertex AI模型监控`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 训练一个`AutoML`模型。
- 将`模型`资源部署到`端点`资源。
- 配置`端点`资源进行模型监控。
- 为偏差生成合成预测请求。
- 等待电子邮件提醒通知。
- 为漂移生成合成预测请求。
- 等待电子邮件提醒通知。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您使用的数据集版本仅使用年份、月份和日期字段来预测每日平均温度值（mean_temp）。

成本

此教程使用Google Cloud的付费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用情况生成成本估算。

### 设立本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行这个笔记本的所有要求。您可以跳过这一步。

**否则**，请确保您的环境满足本笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行Jupyter笔记本

Google Cloud关于[建立Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简化的指南：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口中运行`pip install jupyter`。

5. 要启动Jupyter，在终端窗口中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开这个笔记本。

## 安装

安装执行此笔记本所需的包。

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform \
                                             google-cloud-bigquery

### 重启内核

在安装完SDK后，您需要重新启动笔记本内核，以便它可以找到这些软件包。您可以通过 *Kernel -> Restart Kernel* 来重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。创建账户时，您将获得$300的免费信用用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并且Python变量以`$`为前缀插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改 `REGION` 变量，在本文档的其余部分中使用。以下是 Vertex AI 支持的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

**对于本文档，我们建议您将地区保持默认值 `us-central1`**。

您不能在 Vertex AI 中使用多地区存储桶进行训练。并非所有地区都支持所有 Vertex AI 服务。

了解有关[Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您参加现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户之间在创建的资源上发生名称冲突，您为每个实例会话创建一个 UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

用户电子邮件

将您的用户电子邮箱地址设置为接收监控警报。

In [ ]:
USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

### 验证您的 Google Cloud 帐号

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经通过身份验证。

**如果您正在使用 Colab**，请运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

1. 在 Cloud 控制台中，转到 [**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 单击 **创建服务帐号**。

3. 在 **服务帐号名称**字段中输入一个名称，然后单击 **创建**。

4. 在 **授予此服务帐号对项目的访问权限**部分，单击 **角色** 下拉列表。在筛选框中键入 "Vertex AI"，然后选择 **Vertex AI 管理员**。在筛选框中键入 "Storage Object Admin"，然后选择 **Storage Object Admin**。

5. 单击 **创建**。包含您密钥的 JSON 文件将下载到您的本地环境。

6. 在下面的单元格中，将您的服务帐号密钥路径作为 `GOOGLE_APPLICATION_CREDENTIALS` 变量输入，并运行该单元格。

### 登录您的谷歌云账户并启用人工智能服务

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 关于服务账户和权限的注意事项

**默认情况下不需要任何配置**，如果遇到任何权限相关问题，请确保上述服务账户具有所需的角色：


|服务账户邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine默认服务账户|Dataflow管理员，Dataflow Worker，存储管理员，BigQuery管理员，Vertex AI用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI平台服务代理|Vertex AI服务代理|

1. 进入 https://console.cloud.google.com/iam-admin/iam。
2. 选中"包括由Google提供的角色授予"复选框。
3. 查找上述邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请给这两个服务账户授予"BigQuery数据查看器"角色。
- 对于CSV数据源，请给这两个服务账户授予"存储对象查看器"角色。

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

在本教程中，您将使用与用于训练预训练模型的相同公共BigQuery表中的数据。您将创建一个客户端接口，然后用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

## Vertex AI模型监控简介

Vertex AI模型监控支持AutoML表格式模型和自定义表格式模型。您可以监控入站预测请求中特征的倾斜和漂移检测，或者出站预测响应中特征归因（可解释的AI）的倾斜和漂移检测--即归因对输出（预测）的贡献分布。

以下是启用模型监控的基本步骤：

1. 将`Vertex AI` AutoML或自定义表格式模型部署到`Vertex AI Endpoint`。
2. 配置模型监控规范。
3. 将模型监控规范上传到`Vertex AI Endpoint`。
4. 上传或自动生成用于解析的`输入模式`。
5. 对于特征倾斜检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的`Vertex AI Explainability`规范。

配置后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控时，采样的传入预测请求将被记录到BigQuery表中。然后分析记录请求中包含的输入特征值，以便在指定的时间间隔基础上检查是否倾斜或漂移。您可以设置采样率以监控模型的一部分生产输入，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这被称为输入模式。对于AutoML表格式模型，输入模式会自动生成。对于自定义表格式模型，服务会尝试从前1000个预测请求中自动推导输入模式。或者，您可以上传输入模式。

对于倾斜检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格式模型，这是自动推导的。对于自定义表格式模型，您需要将训练数据上传到服务，并让服务自动推导分布。

对于特征归因的倾斜和漂移检测，需要为自定义表格式模型启用`Vertex AI Explainability`。对于AutoML模型，`Vertex AI Explainability`会自动启用。

了解有关[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)的更多信息。

大数据查询训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 创建数据集

#### BigQuery输入数据

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `bq_source`：将数据项从BigQuery表导入到`Dataset`资源中。
- `labels`：用户定义的元数据。在这个示例中，您存储包含用户定义数据的Cloud Storage存储桶的位置。

了解有关[从BigQuery表创建TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)的更多信息。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="gsod_" + UUID, bq_source=[IMPORT_FILE]
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行这个管道。

#### 创建训练管道

通过`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，参数如下：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用到输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod_" + UUID,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

#### 运行训练流水线

接下来，通过调用方法`run`并传入以下参数来运行创建的DAG以启动训练任务：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则训练可能在服务认为无法进一步改善模型目标测量之前提前完成。

完成后，`run`方法将返回`Model`资源。

训练流水线的执行可能需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并传入以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

### 配置警报规范

首先，您需要使用以下设置配置`alerting_config`规范：

- `user_emails`：要发送警报的一个或多个电子邮件列表。
- `enable_logging`：将检测到的异常流到云日志。默认为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您可以使用以下设置配置`schedule_config`规范：

- `monitor_interval`：设置模型监控作业的调度间隔时间（单位为小时）。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置采样规范

接下来，您使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`: 作为百分比的采样率（介于0和1之间），用于随机采样用于监视的预测请求。选择的样本将被记录到BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

配置漂移检测规范

接下来，您可以使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对的字典，其中键是要监视漂移的输入特征，值是检测阈值。如果未指定，默认特征的漂移阈值为0.3（30％）。

*注意:* 启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {"year": DRIFT_THRESHOLD_VALUE, "motnth": DRIFT_THRESHOLD_VALUE}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置偏差检测规范

接下来，您可以使用以下设置配置`skew_config`规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为BigQuery表。否则，设置`data_format`必须设置为以下值之一。数据的位置必须是Cloud Storage位置。
  - `csv`：
  - `jsonl`：
  - `tf-record`：
- `skew_thresholds`：一个键值对字典，其中键是要监视偏差的输入特征。值是检测阈值。如果未指定，则特征的默认偏差阈值为0.3（30%）。
- `target_field`：训练数据集的目标标签

*注意：* 启用偏差检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://" + BQ_TABLE
# Prediction target column name in training dataset.
TARGET = label_column

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "year": SKEW_THRESHOLD_VALUE,
    "month": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装客观规格

最后，您需使用以下设置组装客观规格`objective_config`：

- `skew_detection_config`：（可选）用于偏斜检测配置的规格。
- `drift_detection_config`：（可选）用于漂移检测配置的规格。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型，这被称为“输入模式”。

对于“AutoML”模型，输入模式是预定义的，并由监控服务自动加载。

创建监控任务

您可以使用 `aiplatform.ModelDeploymentMonitoringJob.create()` 方法，根据您的监控规范创建监控任务，并传入以下参数：

- `display_name`: 监控任务的可读名称。
- `project`: 项目ID。
- `region`: 区域。
- `endpoint`: 要启用监控的 `Vertex AI Endpoint` 的完全限定资源名称。
- `logging_sampling_strategy`: 采样配置的规范。
- `schedule_config`: 调度配置的规范。
- `alert_config`: 警报配置的规范。
- `objective_configs`: 目标配置的规范。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job)

#### 监控作业的电子邮件通知。

电子邮件通知将发送到警报配置中的电子邮件地址，通知模型监控作业现已启用。

内容将如下所示：

您好，Vertex AI客户，

您收到此邮件是因为您正在使用Vertex AI模型监控服务。
本邮件是通知您我们已收到您设置预测终端漂移或偏倚检测的请求。从现在开始，传入的预测请求将被抽样并记录以供分析。
原始请求和响应将从预测服务中收集，并保存在bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict。

监控作业状态

在启动“Vertex AI模型监控”作业后，直到计算“偏斜分布基线”之前，它将处于“挂起”状态。监控服务会启动一个批处理作业，从训练数据中生成分布基线。

一旦基线分布生成，监控作业将进入“离线”状态。按照每个时间间隔的基础，例如每小时一次，监控作业将在分析采样数据时进入“运行”状态。完成后，它将返回到“离线”状态，等待下次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=churn_{UUID}")
job = jobs[0]
print(job.state)

### 自动生成基线分布

接下来，监控服务将创建一个批处理作业来分析训练数据，生成基线分布。一旦完成，监控服务将开始在指定间隔上进行监控。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(180)

生成用于偏斜检测的合成预测请求

接下来，您从BigQuery训练表中提取前1000个实例用于预测请求。您修改数据（合成数据）以触发预测请求中的偏斜检测，训练分布与服务分布相比如下所示：

- `year`：将所有值都设置为3（原来是2）。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "year":
            value = "3"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

###发出预测请求

接下来，您可以使用`predict()`方法将1000个预测请求发送给您的`Vertex AI Endpoint`资源。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

监控服务启动后，抽样的预测请求将被记录到云存储中。在下一个监控间隔，抽样的预测将被复制到BigQuery日志表中。一旦条目出现在BigQuery表中，监控服务将分析抽样数据。

接下来，您等待第一个被记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，采样率为50%，您应该看到大约500个条目。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 在监控期间进行偏斜检测

在下一个监控间隔期间将发生特征输入偏斜检测。在本教程中，您将监控间隔设置为一小时。因此，大约在一个小时后，您的监控作业将从“OFFLINE”状态变为“RUNNING”状态。运行时，它将分析此间隔内预测的已记录采样表，并将它们与基准分布进行比较。

分析完成后，监控作业将发送电子邮件通知有关检测到的偏斜，本例中为`year`，并且监控作业将进入“OFFLINE”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔上进行分析到您收到电子邮件警报可能需要多达40分钟。

内容将如下显示：

<blockquote>
    你好，Vertex AI 客户，

您收到此邮件是因为您正在订阅 Vertex AI 模型监控服务。
此邮件仅用于通知您，您部署的模型中检测到了一些异常情况，可能需要您的关注。

基本信息：

终端名称：projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监控作业：projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计和异常根路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery 命令：SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`

训练预测偏斜异常（原始特征）：

异常报告路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关警报的更多信息，请访问模型监控警报页面。

部署模型 id：<deployed-model-id>

特征名称	异常简要说明	异常详细说明
country	训练与服务之间的高 Linfty 距离	        训练和服务之间的 Linfty 距离为0.947563（最多六个有效数字），高于阈值0.5。具有最大差异值的特征为：Year
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 记录抽样请求

在下一个监控间隔内，抽样预测结果将被复制到BigQuery记录表中。一旦条目进入BigQuery表，监控服务将分析抽样数据。

接下来，您需要等待记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，因此您应该看到大约1000条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 505:
        break

### 生成用于漂移检测的合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例用于预测请求。您修改数据（合成数据）以触发在预测请求中从训练分布与服务分布之间的漂移检测，具体操作如下：

- `月份`：将所有值设置为1。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "month":
            value = "1"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您可以使用 `predict()` 方法向您的 `Vertex AI Endpoint` 资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在下一个监控间隔中，抽样的预测将被复制到BigQuery日志表中。一旦条目在BigQuery表中，监控服务将分析抽样数据。

接下来，您等待第一批已记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，您应该会看到大约1500条目。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 1050:
        break

### 监控过程中的漂移检测

下一个监控间隔将进行特征输入漂移检测。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行时，它将分析在此间隔期间预测中记录的样本表，并将其与先前监控间隔的分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的漂移，例如`cnt_user_engagement`，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔发生分析到您收到电子邮件通知警报可能需要多达40分钟。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署并删除“Vertex AI Endpoint”资源

您可以使用“delete（）”方法删除您的“Vertex AI Endpoint”资源。在删除之前，部署到您的“Vertex AI Endpoint”资源的任何模型必须先取消部署。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

清理工作

要清理本项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源。

In [ ]:
! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}